In [2]:
import torch
import tkinter as tk
from tkinter import scrolledtext, messagebox
import threading
from transformers import AutoTokenizer, AutoModelForCausalLM

class LeetCodeClassifierApp:
    def __init__(self, root, model_path="./leetcode-classifier-model"):
        self.root = root
        self.root.title("LeetCode Problem Classifier")
        self.root.geometry("800x600")
        self.root.minsize(600, 500)
        
        # Path to model
        self.model_path = model_path
        
        # Check for CUDA availability
        self.cuda_available = torch.cuda.is_available()
        self.device = "cuda" if self.cuda_available else "cpu"
        
        # Status variable
        self.load_model_status = tk.StringVar()
        self.load_model_status.set(f"Status: Loading model on {'GPU (CUDA)' if self.cuda_available else 'CPU'}...")
        
        # UI elements
        self.create_widgets()
        
        # Load model in a separate thread
        self.model = None
        self.tokenizer = None
        threading.Thread(target=self.load_model).start()
        
        # Categories list
        self.categories = [
            "Array", "Dynamic Programming", "String", "Math", "Tree", 
            "Depth-first Search", "Greedy", "Hash Table", "Binary Search", 
            "Breadth-first Search", "Sort", "Two Pointers", "Backtracking", 
            "Stack", "Design"
        ]
    
    def create_widgets(self):
        # Main frame to contain everything
        main_frame = tk.Frame(self.root)
        main_frame.pack(fill='both', expand=True, padx=10, pady=10)
        
        # Title
        title_label = tk.Label(main_frame, text="LeetCode Problem Classifier", font=("Arial", 18, "bold"))
        title_label.pack(pady=10)
        
        # GPU status indicator
        gpu_status = "GPU ENABLED" if self.cuda_available else "CPU MODE"
        gpu_color = "#4CAF50" if self.cuda_available else "#FFA500"
        gpu_label = tk.Label(main_frame, text=gpu_status, font=("Arial", 10, "bold"),
                            fg="white", bg=gpu_color, padx=10, pady=5)
        gpu_label.pack(pady=5)
        
        # Instructions
        instructions = tk.Label(main_frame, text="Enter a problem description and click 'Classify' to determine the problem type")
        instructions.pack(pady=5)
        
        # Status label
        status_label = tk.Label(main_frame, textvariable=self.load_model_status)
        status_label.pack(pady=5)
        
        # Text input area
        input_label = tk.Label(main_frame, text="Problem Description:")
        input_label.pack(anchor='w', padx=10)
        
        self.text_input = scrolledtext.ScrolledText(main_frame, wrap=tk.WORD, height=15)
        self.text_input.pack(fill='both', expand=True, padx=10, pady=10)
        
        # Example buttons
        examples_frame = tk.Frame(main_frame)
        examples_frame.pack(fill='x', padx=10)
        
        examples_label = tk.Label(examples_frame, text="Examples:")
        examples_label.pack(anchor='w')
        
        examples_buttons_frame = tk.Frame(examples_frame)
        examples_buttons_frame.pack(fill='x', pady=5)
        
        examples = {
            "Two Sum": "Given an array of integers nums and an integer target, return indices of the two numbers such that they add up to target.",
            "Valid Parentheses": "Given a string s containing just the characters '(', ')', '{', '}', '[' and ']', determine if the input string is valid.",
            "Merge Sorted Lists": "You are given the heads of two sorted linked lists list1 and list2. Merge the two lists into one sorted list."
        }
        
        col = 0
        for title, description in examples.items():
            btn = tk.Button(examples_buttons_frame, text=title, 
                            command=lambda desc=description: self.set_example(desc))
            btn.grid(row=0, column=col, padx=5)
            col += 1
        
        # Result display
        result_label = tk.Label(main_frame, text="Result:", font=("Arial", 12))
        result_label.pack(anchor='w', padx=10, pady=(10, 0))
        
        self.result_display = tk.Label(main_frame, text="No classification yet", font=("Arial", 14), 
                                      bg="#f0f0f0", relief="ridge", height=2)
        self.result_display.pack(fill='x', padx=10, pady=5)
        
        # Classify button
        self.classify_button = tk.Button(main_frame, text="CLASSIFY PROBLEM", command=self.classify_problem, 
                                     bg="#4CAF50", fg="white", font=("Arial", 14, "bold"), 
                                     height=2, relief="raised", borderwidth=3)
        self.classify_button.pack(fill='x', padx=50, pady=20)
        
        # Disable classify button until model is loaded
        self.classify_button.config(state=tk.DISABLED)
    
    def load_model(self):
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)
            
            # Load model and move to GPU if available
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_path, 
                torch_dtype=torch.float16 if self.cuda_available else torch.float32
            )
            
            # Move model to GPU if available
            if self.cuda_available:
                self.model = self.model.to("cuda")
                
            device_info = f"GPU (CUDA)" if self.cuda_available else "CPU"
            self.load_model_status.set(f"Status: Model loaded successfully on {device_info}!")
            self.classify_button.config(state=tk.NORMAL)
        except Exception as e:
            self.load_model_status.set(f"Error loading model: {str(e)}")
            messagebox.showerror("Error", f"Failed to load model: {str(e)}")
    
    def set_example(self, description):
        self.text_input.delete(1.0, tk.END)
        self.text_input.insert(tk.END, description)
    
    def classify_problem(self):
        if not self.model or not self.tokenizer:
            messagebox.showwarning("Warning", "Model is not loaded yet")
            return
        
        problem_description = self.text_input.get(1.0, tk.END).strip()
        if not problem_description:
            messagebox.showwarning("Warning", "Please enter a problem description")
            return
        
        self.classify_button.config(state=tk.DISABLED, text="Classifying...")
        self.load_model_status.set("Status: Classifying problem...")
        
        # Run classification in a separate thread to avoid freezing the UI
        threading.Thread(target=self.run_classification, args=(problem_description,)).start()
    
    def run_classification(self, problem_description):
        try:
            prompt = f"""Given the following programming problem, classify it into one of these categories:
Array, Dynamic Programming, String, Math, Tree, Depth-first Search, Greedy, Hash Table, Binary Search, Breadth-first Search, Sort, Two Pointers, Backtracking, Stack, Design

Problem description:
{problem_description}

The problem type is:"""
            
            # Create inputs and move to CUDA if available
            inputs = self.tokenizer(prompt, return_tensors="pt")
            
            if self.cuda_available:
                # Move inputs to GPU
                inputs = {k: v.to("cuda") for k, v in inputs.items()}
            
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=10,
                    temperature=0.1,
                    num_return_sequences=1,
                    pad_token_id=self.tokenizer.eos_token_id
                )
            
            generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            predicted_category = generated_text.split("The problem type is:")[-1].strip()
            
            # Match to closest category
            for category in self.categories:
                if category.lower() in predicted_category.lower():
                    predicted_category = category
                    break
            
            # Update UI on the main thread
            self.root.after(0, self.update_result, predicted_category)
            
        except Exception as e:
            self.root.after(0, self.show_error, str(e))
    
    def update_result(self, result):
        self.result_display.config(text=f"Problem Type: {result}")
        self.classify_button.config(state=tk.NORMAL, text="CLASSIFY PROBLEM")
        self.load_model_status.set("Status: Ready")
    
    def show_error(self, error_msg):
        messagebox.showerror("Error", f"Classification failed: {error_msg}")
        self.classify_button.config(state=tk.NORMAL, text="CLASSIFY PROBLEM")
        self.load_model_status.set("Status: Ready")

if __name__ == "__main__":
    root = tk.Tk()
    app = LeetCodeClassifierApp(root)
    root.mainloop()

KeyboardInterrupt: 